In [ ]:
import keras.utils
import numpy as np
import tqdm.notebook
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten

inp = Input(shape=(240, 320, 1))

n_filters = 512
max_p = inp

for _ in range(3):
    conv = Conv2D(filters=n_filters, kernel_size=5, activation='relu')(max_p)
    max_p = MaxPooling2D(pool_size=(2, 2))(conv)
    
    n_filters //= 2

conv = Conv2D(filters=n_filters, kernel_size=3, activation='relu')(max_p)
max_p = MaxPooling2D(pool_size=(2, 2))(conv)

flatten = Flatten()(max_p)
dense = Dense(128, activation='relu')(flatten)
# out = Dense(14, activation='relu')(dense)

head_out = Dense(1, activation='relu', name='head_out')(dense)
neck_out = Dense(1, activation='relu', name='neck_out')(dense)
shoulder_to_shoulder_out = Dense(1, activation='relu', name='shoulder_to_shoulder_out')(dense)
arm_out = Dense(1, activation='relu', name='arm_out')(dense)
shoulder_to_wrist_out = Dense(1, activation='relu', name='shoulder_to_wrist_out')(dense)
torso_out = Dense(1, activation='relu', name='torso_out')(dense)
bicep_out = Dense(1, activation='relu', name='bicep_out')(dense)
wrist_out = Dense(1, activation='relu', name='wrist_out')(dense)
chest_out = Dense(1, activation='relu', name='chest_out')(dense)
waist_out = Dense(1, activation='relu', name='waist_out')(dense)
pelvis_out = Dense(1, activation='relu', name='pelvis_out')(dense)
leg_out = Dense(1, activation='relu', name='leg_out')(dense)
inner_leg_out = Dense(1, activation='relu', name='inner_leg_out')(dense)
thigh_out = Dense(1, activation='relu', name='thigh_out')(dense)
knee_out = Dense(1, activation='relu', name='knee_out')(dense)
calf_out = Dense(1, activation='relu', name='calf_out')(dense)


model = Model(
    inputs=inp,
    outputs=[
    head_out,
    neck_out,
    shoulder_to_shoulder_out,
    arm_out,
    shoulder_to_wrist_out,
    torso_out,
    bicep_out,
    wrist_out,
    chest_out,
    waist_out,
    pelvis_out,
    leg_out,
    inner_leg_out,
    thigh_out,
    knee_out,
    calf_out
    ],
    name='conv_bodies'
)

#model.save_weights("default.weights.h5")

#model.summary()

In [ ]:
# from keras.utils import plot_model
# 
# plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, show_layer_activations=True)

In [ ]:
#FOR BODY_M DATASET
# import csv
# 
# with open('../bodym-dataset/train/subject_to_photo_map.csv', 'r') as f:
#     reader = csv.reader(f)
#     uuid_raw = list(reader)
# from collections import defaultdict
# 
# uuid_map = defaultdict(list)
# 
# for subject_id, photo_id in uuid_raw[1:]:
#     uuid_map[subject_id].append(photo_id)
# image_data = {}
# 
# with open('../bodym-dataset/train/measurements.csv', 'r') as f:
#     reader = csv.reader(f)
#     
#     for line in reader:
#         subject_id = line[0]
#         
#         
#         for image_id in uuid_map[subject_id]:
#             image_data[image_id] = np.array(list(map(float, line[1:])))
#             
# import os
# 
# import cv2
# from tqdm import tqdm
# 
# raw_dataset = []
# 
# for image_id in tqdm(image_data):
#     img = cv2.imread(os.path.join('../bodym-dataset/train/mask', image_id + '.png'), cv2.IMREAD_GRAYSCALE)
#     img = cv2.resize(img, (180, 240))
#     
#     raw_dataset.append((
#         np.expand_dims(img, 2), 
#         image_data[image_id])
#     )

In [ ]:
import os
import cv2
import random
from tqdm import tqdm_notebook

class Database_Loader(keras.utils.Sequence):
    
    def __init__(self, image_location, data_location, sample_count, batch_size, shuffle=True, seed=0, input_dimensions=(240, 320), prefix="Avatar_", use_memory=False, load_data = True, random_sample = False):
        self.image_location = image_location
        self.data_location = data_location
        self.sample_count = sample_count
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.seed = seed
        self.input_dimensions = input_dimensions
        self.prefix = prefix
        self.use_memory = use_memory
        self.load_data = load_data
        self.random_sample = random_sample

        if self.random_sample:
            self.IDs = random.sample(range(len(os.listdir(self.image_location))),self.sample_count)
        else:
            self.IDs = [x for x in range(self.sample_count)]
            self._load_data()

        super().__init__(workers=2, use_multiprocessing=True)
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(self.sample_count / self.batch_size))
    
    
    def _load_data(self):
        if self.load_data:
            self.data = dict()
            for index in self.IDs:
                self.data[index] = np.load(os.path.join(self.data_location + self.prefix + f"{index:06d}.npy"))

        if self.use_memory:
            self.images = dict()
            for index in self.IDs:
                self.images[index] = cv2.imread(os.path.join(self.image_location + self.prefix + f"{index:06d}.png"), cv2.IMREAD_GRAYSCALE)
    
    def __getitem__(self, index):
        X = np.empty(shape=(self.batch_size, 240, 320))
        y = {
            'head_out': [],
            'neck_out': [],
            'shoulder_to_shoulder_out': [],
            'arm_out': [],
            'shoulder_to_wrist_out': [],
            'torso_out': [],
            'bicep_out': [],
            'wrist_out': [],
            'chest_out': [],
            'waist_out': [],
            'pelvis_out': [],
            'leg_out': [],
            'inner_leg_out': [],
            'thigh_out': [],
            'knee_out': [],
            'calf_out': []
        }
        
        start_index = index * self.batch_size + 1
        
        for i in range(self.batch_size):
            name = f"{self.IDs[(start_index + i)%self.sample_count]:06d}"
            
            if self.load_data:
                current_measurement = self.data[self.IDs[(start_index + i)%self.sample_count]][:-1]          
            else:
                current_measurement = np.load(os.path.join(self.data_location + self.prefix + name + ".npy"))[:-1]
                
            if self.use_memory:
                X[i,] = self.images[self.IDs[(start_index + i)%self.sample_count]]
                
            else:
                X[i,] = cv2.imread(os.path.join(self.image_location + self.prefix + name + ".png"), cv2.IMREAD_GRAYSCALE)

            y['head_out'].append([current_measurement[0]])
            y['neck_out'].append([current_measurement[1]])
            y['shoulder_to_shoulder_out'].append([current_measurement[2]])
            y['arm_out'].append([current_measurement[3]])
            y['shoulder_to_wrist_out'].append([current_measurement[4]])
            y['torso_out'].append([current_measurement[5]])
            y['bicep_out'].append([current_measurement[6]])
            y['wrist_out'].append([current_measurement[7]])
            y['chest_out'].append([current_measurement[8]])
            y['waist_out'].append([current_measurement[9]])
            y['pelvis_out'].append([current_measurement[10]])
            y['leg_out'].append([current_measurement[11]])
            y['inner_leg_out'].append([current_measurement[12]])
            y['thigh_out'].append([current_measurement[13]])
            y['knee_out'].append([current_measurement[14]])
            y['calf_out'].append([current_measurement[15]])

        for key, value in y.items():
            y[key] = np.array(value)
            
        return X, y
    
    def on_epoch_end(self):
        if self.random_sample:
            self.IDs = random.sample(range(len(os.listdir(self.image_location))),self.sample_count)
            self._load_data()
        
        if self.shuffle:
            np.random.seed(self.seed)
            np.random.shuffle(self.IDs)
        else:
            self.IDs = np.arange(self.sample_count)
            
    
    
        

In [ ]:
# import cv2
# import os
# from tqdm import tqdm
# import tensorflow as tf
# 
# avatar_count = 79999
# image_folder_path = os.path.join("../Surreact-APose/train/imgs_nobg_frontEDITED/")
# 
# 
#     
# 
# 
# def load_sample():
#     for number in range(avatar_count):
#         if number == 5:
#             continue
#         current_measurement = np.load(f"../Surreact-APose/train/measurements/Avatar_{number:06d}.npy")
#         current_image = cv2.imread(f"../Surreact-APose/train/imgs_nobg_frontEDITED/Avatar_{number:06d}.png", cv2.IMREAD_GRAYSCALE)
#         measurements = dict()
#         measurements['head_out'] = current_measurement[0]
#         measurements['neck_out'] = current_measurement[1]
#         measurements['shoulder_to_shoulder_out'] = current_measurement[2]
#         measurements['arm_out'] = current_measurement[3]
#         measurements['shoulder_to_wrist_out'] = current_measurement[4]
#         measurements['torso_out'] = current_measurement[5]
#         measurements['bicep_out'] = current_measurement[6]
#         measurements['wrist_out'] = current_measurement[7]
#         measurements['chest_out'] = current_measurement[8]
#         measurements['waist_out'] = current_measurement[9]
#         measurements['pelvis_out'] = current_measurement[10]
#         measurements['leg_out'] = current_measurement[11]
#         measurements['inner_leg_out'] = current_measurement[12]
#         measurements['thigh_out'] = current_measurement[13]
#         measurements['knee_out'] = current_measurement[14]
#         measurements['calf_out'] = current_measurement[15]
#     
#         yield current_image, measurements
# 
# 
# train_data = tf.data.Dataset.from_generator(load_sample, output_types=(tf.float32, {"head_out": tf.float32, "neck_out": tf.float32, "shoulder_to_shoulder_out": tf.float32, "arm_out": tf.float32, "shoulder_to_wrist_out": tf.float32, "torso_out": tf.float32, "bicep_out": tf.float32, "wrist_out": tf.float32, "chest_out": tf.float32, "waist_out": tf.float32, "pelvis_out": tf.float32, "leg_out": tf.float32, "inner_leg_out": tf.float32, "thigh_out": tf.float32, "knee_out": tf.float32, "calf_out": tf.float32}), output_shapes=((None, None), {"head_out": [], "neck_out": [], "shoulder_to_shoulder_out": [], "arm_out": [], "shoulder_to_wrist_out": [], "torso_out": [], "bicep_out": [], "wrist_out": [], "chest_out": [], "waist_out": [], "pelvis_out": [], "leg_out": [], "inner_leg_out": [], "thigh_out": [], "knee_out": [], "calf_out": []}))
# train_data = train_data.batch(5)



# train_data = keras.utils.image_dataset_from_directory(
#     directory="../Surreact-APose/train/imgs_nobg_frontEDITED/",
#     labels=None,
#     batch_size=32,
#     image_size=(240, 320),
#     shuffle=False, da]
#     validation_split=0.2,
#     color_mode="grayscale",
#     subset="both",
#     seed=69
# )´


In [ ]:
quickTrain = {'image_location': "../Surreact-APose/train/imgs_nobg_frontEDITED/", 
              'data_location': "../Surreact-APose/train/measurements/", 
              'sample_count': 1, 
              'batch_size': 1, 
              'seed': 69, #np.random.randint(0, 10000), 
              'use_memory': False,
              'random_sample': False}

quickValidate = {'image_location': "../Surreact-APose/train/imgs_nobg_frontEDITED/",
              'data_location': "../Surreact-APose/train/measurements/",
              'sample_count': 1,
              'batch_size': 1,
              'seed': 69, #np.random.randint(0, 10000),
              'use_memory': False,
              'random_sample': False}

train_generator = Database_Loader(**quickTrain)
validation_generator = Database_Loader(**quickValidate)







# X = np.array([data[0] for data in dataset.values()])
# y_true = np.array([data[1] for data in dataset.values()])
# 
# (
#     y_true_head, y_true_neck, y_true_sts,y_true_arm, y_true_stw, y_true_torso, y_true_bicep, y_true_wrist, y_true_chest, y_true_waist, y_true_pelvis, y_true_leg, y_true_inner_leg, y_true_thigh, y_true_knee, y_true_calf, height
# ) = np.hsplit(y_true, 17) #17 is height


In [ ]:
# from keras.models import load_model
# 
# model = load_model('')

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.losses import MeanSquaredError
from keras_tqdm import TQDMNotebookCallback

model.load_weights("default.weights.h5")

model.compile(
    optimizer=Adam(),
    loss={
        'head_out': MeanSquaredError(),
        'neck_out': MeanSquaredError(),
        'shoulder_to_shoulder_out': MeanSquaredError(),
        'arm_out': MeanSquaredError(),
        'shoulder_to_wrist_out': MeanSquaredError(),
        'torso_out': MeanSquaredError(),
        'bicep_out': MeanSquaredError(),
        'wrist_out': MeanSquaredError(),
        'chest_out': MeanSquaredError(),
        'waist_out': MeanSquaredError(),
        'pelvis_out': MeanSquaredError(),
        'leg_out': MeanSquaredError(),
        'inner_leg_out': MeanSquaredError(),
        'thigh_out': MeanSquaredError(),
        'knee_out': MeanSquaredError(),
        'calf_out': MeanSquaredError()
    },
    loss_weights={
        'head_out': 2.0,
        'neck_out': 1.0,
        'shoulder_to_shoulder_out': 2.0,
        'arm_out': 1.0,
        'shoulder_to_wrist_out': 1.0,
        'torso_out': 1.0,
        'bicep_out': 1.0,
        'wrist_out': 1.0,
        'chest_out': 1.0,
        'waist_out': 1.0,
        'pelvis_out': 1.0,
        'leg_out': 1.0,
        'inner_leg_out': 3.0,
        'thigh_out': 1.0,
        'knee_out': 1.0,
        'calf_out': 1.0
    },
    # loss='mse',
    metrics={
        'head_out': ['mae', 'mse'],
        'neck_out': ['mae', 'mse'],
        'shoulder_to_shoulder_out': ['mae','mse'],
        'arm_out': ['mae','mse'],
        'shoulder_to_wrist_out': ['mae','mse'],
        'torso_out': ['mae','mse'],
        'bicep_out': ['mae','mse'],
        'wrist_out': ['mae','mse'],
        'chest_out': ['mae','mse'],
        'waist_out': ['mae','mse'],
        'pelvis_out': ['mae','mse'],
        'leg_out': ['mae','mse'],
        'inner_leg_out': ['mae','mse'],
        'thigh_out': ['mae','mse'],
        'knee_out': ['mae','mse'],
        'calf_out': ['mae','mse']
    },
)
keras.utils.set_random_seed(42)


checkpoint_filepath = './models/checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True)


model.fit(
    x=train_generator,
    validation_data=validation_generator,
    callbacks=[
        
        TensorBoard(write_graph=False,log_dir="./logs"),
        # TQDMNotebookCallback(separator="", metric_format="{name}: {value:0.8f} || ")
        #model_checkpoint_callback
    ],
    batch_size=1,
    epochs=2000,
    verbose=1
)

# 
# model.fit(
#     x=X,
#     y={
#         'head_out': y_true_head,
#         'neck_out': y_true_neck,
#         'shoulder_to_shoulder_out': y_true_sts,
#         'arm_out': y_true_arm,
#         'shoulder_to_wrist_out': y_true_stw,
#         'torso_out': y_true_torso,
#         'bicep_out': y_true_bicep,
#         'wrist_out': y_true_wrist,
#         'chest_out': y_true_chest,
#         'waist_out': y_true_waist,
#         'pelvis_out': y_true_pelvis,
#         'leg_out': y_true_leg,
#         'inner_leg_out': y_true_inner_leg,
#         'thigh_out': y_true_thigh,
#         'knee_out': y_true_knee,
#         'calf_out': y_true_calf
#     },
#     batch_size=8,
#     epochs=10,
#     validation_data=train_data[1],
#     shuffle=True,
#     callbacks=[TensorBoard(write_graph=False)]
# )

In [ ]:
    #Data modifiaction

# import os
# from tqdm import tqdm
# 
# avatar_count = 79999
# counter = 0
# 
# for number in tqdm(range(avatar_count)):
#     if number == 5:
#         continue
#     file = (tuple(np.load(f"../Surreact-APose/train/measurements/Avatar_{number:06d}.npy", allow_pickle=True)))
#     measurements = []
#     for value in file:
#         measurements.append(float(value))
#     np.save(f"../Surreact-APose/train/measurements/Avatar_{counter:06d}", measurements)
#     counter += 1

In [ ]:
#Image modification

# import os
# import cv2
# from tqdm import tqdm
# 
# 
# avatar_count = 79999
# counter = 0
# 
# for number in tqdm(range(avatar_count)):
#     if number == 5:
#         continue
#     file = cv2.imread(f"../Surreact-APose/train/imgs_nobg_front/Avatar_{number:06d}.png", cv2.IMREAD_UNCHANGED)
#     file = cv2.cvtColor(file, cv2.COLOR_mRGBA2RGBA)
#     ret, binary = cv2.threshold(file, 0, 255, 0)
# 
#     cv2.imwrite(f"../Surreact-APose/train/imgs_nobg_frontEDITED/Avatar_{counter:06d}.png", binary)
    counter +=1

In [ ]:
# from tensorboard.backend.event_processing import event_accumulator
# import pandas as pd
# 
# 
# def parse_tensorboard(path, scalars):
#     """returns a dictionary of pandas dataframes for each requested scalar"""
#     ea = event_accumulator.EventAccumulator(
#         path,
#         size_guidance={event_accumulator.SCALARS: 0},
#     )
#     _absorb_print = ea.Reload()
#     print(ea.Tags()["tensors"])
#     # make sure the scalars are in the event accumulator tags
#     assert all(
#         s in ea.Tags()["tensors"] for s in scalars
#     ), "some scalars were not found in the event accumulator"
#     return {k: pd.DataFrame(ea.Tensors(k)) for k in scalars}
# 
# 
# 
# a = parse_tensorboard("logsTest/train/events.out.tfevents.1711732161.root.21184.39.v2",['epoch_head_out_mae', 'batch_head_out_mae'])
# 
# 
# print(a)